In [ ]:
%cd /Users/sophiatonelli/library_script/script

import os
os.chdir('/Users/sophiatonelli/library_script/script/work')


import os, sys # to communicate with the machine you are operating
sys.path.append(os.getcwd()) #  to search python modules from the current path. This helps us to use SCRIPT modules.
from __future__ import print_function # Force python2 to generate same output as python3 when using print function
import numpy as np # for doing numerical calcuations
import matplotlib.pyplot as plt # for making and displaying plots
import script # for using SCRIPT functionalities
import matplotlib
%matplotlib inline
matplotlib.rcParams.update({'font.size': 11})# increase fontsize of text on plots

In [ ]:
gadget_snap = '/Users/sophiatonelli/Downloads/snap_140' #import data file 
#'/Users/sophiatonelli/Downloads/snap_120' 
outpath = '/Users/sophiatonelli/library_script/script/work/script_files' #./script_files' # the generated density and velocity fields will be saved in script_files/
scaledist = 1e-3 # sets the scale for box size. 1e-3 for box of length in kpc/h and 1. for length in cMpc/h

default_simulation_data = script.default_simulation_data(gadget_snap, outpath, sigma_8=0.829, ns=0.961, omega_b=0.0482, scaledist=scaledist) # Read binary data files.

In [ ]:
print("Simulation box size:", default_simulation_data.box, "cMpc/h") #box size in cMpc/h
print("Simulation redshift:", default_simulation_data.z) #redshift of the simulation
print("Simulation box size in Mpc/h:", default_simulation_data.box * 1e-3) #box size in Mpc/h


In [ ]:
#define RESOLUTION, number of cells to which density fields are smoothened (i.e. resolution 256/128 cMpc/h)
ngrid = 128  

matter_fields = script.matter_fields(default_simulation_data, ngrid, outpath, overwrite_files=False) 

In [ ]:
#log10Mmin = 9.0 #minimum mass of halo for efficient star formation, Only halos above a certain mass threshold are considered for star formation.
#fcoll_arr = matter_fields.get_fcoll_for_Mmin(log10Mmin) #calculate collapse fraction of each cell
#zeta = 12 #ionizing efficiency parameter, i.e. number of ionizing photons per collapsed baryon

In [ ]:
list_log10Mmin = [7.0, 8.0, 8.5, 9.5, 10.0, 11.0]  #minimum halo masses for star formation
list_zeta = [10, 20, 30, 40]  #ionizing efficiency parameters

nrows, ncols = len(list_log10Mmin), len(list_zeta)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4 * nrows), constrained_layout=True)

for i, log10Mmin in enumerate(list_log10Mmin):
    fcoll_arr = matter_fields.get_fcoll_for_Mmin(log10Mmin)
    print("log10(Mmin):", log10Mmin, "fcoll:", fcoll_arr.mean())

    for j, zeta in enumerate(list_zeta):
        print("zeta:", zeta)
        ionization_map = script.ionization_map(matter_fields)
        qi_arr = ionization_map.get_qi(zeta * fcoll_arr)
        print("qi_arr mean:", qi_arr.mean())

        ax = axes[i,j]  #indexing for 2d axes array 

        im = ax.imshow(
            1 - qi_arr[:, :, 16],
            extent=[0, default_simulation_data.box, 0, default_simulation_data.box],
            cmap='viridis'
        )
        ax.set_title(
        rf"$\log_{{10}}(M_{{min}})={log10Mmin},\ \zeta={zeta}$",
        fontsize=14,
        pad=14)
        ax.set_xticks([])
        ax.set_yticks([])

        ax.text(
        0.5, -0.05,  # x and y position in axes fraction coordinates
        f"$\\langle x_{{\\mathrm{{HI}}}} \\rangle = {1 - qi_arr.mean():.4f}$",
        fontsize=12,
        ha='center',
        va='top',
        transform=ax.transAxes
    )

fig.text(
    0.5, -0.02, rf'$z={default_simulation_data.z:.1f}$',
    fontsize=16,
    ha='center',
    va='top'
)


cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.8, pad=0.02, label=r'$x_{\mathrm{HI}}$', orientation='horizontal')
cbar.ax.tick_params(labelsize=16)
cbar.set_label(r'$x_{\mathrm{HI}}$', fontsize=18, labelpad=12)
#plt.savefig('/Users/sophiatonelli/Desktop/pngs_mpia/label_7to12_10to40z_ionization_maps_redshift8.png', dpi=300, bbox_inches='tight') 
plt.show()


Lower Mmin (e.g. 10⁸ M☉):
More halos contribute.

log10Mmin does not by itself produce ionizing photons — it gates which structures can contribute. The actual ionization strength depends on both fcoll and zeta.
If your simulation doesn't include feedback models, you're implicitly baking feedback effects into Mmin.
​	

Higher collapsed fraction fcoll (fraction of matter in star-forming halos).


####

zeta directly controls how many ionizing photons are produced per baryon in star-forming halos.
It encapsulates:
Star formation efficiency.

You can have a high ζ(zeta) even with low Mmin: if your galaxies are extremely efficient at producing and escaping ionizing photons.


####

Even if fcoll is fixed, increasing zeta boosts the ionized fraction qi !!!!!!!!!!!!!!!!!

This can be physically valid if:

Star formation is bursty and efficient in small halos.
Feedback is delayed or weak.
You're modeling a metal-poor early universe !!


In [ ]:
list_log10Mmin = [8.0, 9.0, 10.0]  #list of minimum halo masses for star formation

#panel grid size (such as 1 row x 5 columns)
nrows, ncols = 1, len(list_log10Mmin)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 4), constrained_layout=True)

#loop through for generating subplots
for i, log10Mmin in enumerate(list_log10Mmin):
    fcoll_arr = matter_fields.get_fcoll_for_Mmin(log10Mmin)
    print("log10(Mmin):", log10Mmin, "fcoll:", fcoll_arr.mean())

    ax = axes[i] if nrows == 1 else axes[i // ncols, i % ncols]
    im = ax.imshow(np.log10(fcoll_arr[:, :, 16]),extent=[0, default_simulation_data.box, 0, default_simulation_data.box],vmin=-3,vmax=0,cmap='magma')
    ax.set_title(f"$\log_{{10}}(M_{{min}})={log10Mmin}$", fontsize=16, pad=10)
    ax.set_xticks([])
    ax.set_yticks([])

    ax.text(
        0.5, -0.05,  # x and y position in axes fraction coordinates
        rf'$\langle f_{{\mathrm{{coll}}}} \rangle = {fcoll_arr.mean():.4f}$',
        fontsize=16,
        ha='center',
        va='top',
        transform=ax.transAxes
    )

fig.text(
    0.5, -0.08, rf'$z={default_simulation_data.z:.1f}$',
    fontsize=16,
    ha='center',
    va='top'
)


cbar = fig.colorbar(im, ax=axes.ravel().tolist(), shrink=0.8, pad=0.02, label=r'$\log f_{\mathrm{coll}}$')
cbar.ax.tick_params(labelsize=16)  
cbar.set_label(r'$\log_{{10}}(f_{\mathrm{coll}})$', fontsize=18, labelpad=12)
#plt.savefig('/Users/sophiatonelli/Desktop/pngs_mpia/fcoll_map_redshift6_rootfinding.png', dpi=300, bbox_inches='tight') 
plt.show()


#plot fcoll field #correlated.

#cmb 
#observable: quasar spectra z=6/5 lyman alpha proof reionization + 21cm line power spectrum
#UV luminsoity function 

Increasing Mmin (Fewer halos contribute) → decreasing / lowering fcoll !!!!!
https://sites.lsa.umich.edu/chrisdessert/wp-content/uploads/sites/560/2018/03/extended-press-schechter4.pdf

also 

Increasing z -> Less time for structure formation !!!!!

In [ ]:
#make a video from the PNG files
"""
import cv2
import os
import glob

#PNG files in order
image_files = ['/Users/sophiatonelli/Desktop/pngs_mpia/script_week2/fcoll_map_redshift8_rootfinding.png', 
               '/Users/sophiatonelli/Desktop/pngs_mpia/script_week2/fcoll_map_redshift7_rootfinding.png',
               '/Users/sophiatonelli/Desktop/pngs_mpia/script_week2/fcoll_map_redshift6_rootfinding.png']

#read the first image to get dimensions
frame = cv2.imread(image_files[0])
height, width, layers = frame.shape

#video writer
out = cv2.VideoWriter('/Users/sophiatonelli/Desktop/pngs_mpia/output_video.mp4', cv2.VideoWriter_fourcc(*'mp4v'), 1, (width, height))


#add each image to the video
for image in image_files:
    frame = cv2.imread(image)
    out.write(frame)

#release the video writer
out.release()

video_path = os.path.abspath('/Users/sophiatonelli/Desktop/pngs_mpia/output_video.mp4')
print(f"✅ Video saved here: {video_path}")

print("Video created successfully!")

"""


✅ Video saved here: /Users/sophiatonelli/Desktop/pngs_mpia/output_video.mp4
Video created successfully!


In [ ]:
list_log10Mmin = [7.0, 8.0, 8.5] #minimum halo masses for star formation #9.5, 10.0, 11.0] 
list_zeta = [10, 30] #, 30, 40]  #ionizing efficiency parameters


nrows, ncols = len(list_log10Mmin), len(list_zeta)
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(4 * ncols, 3 * nrows), constrained_layout=True)


def apply_periodic_index(idx, ngrid): #periodic boundary conditions for DISCRETE integer grid indices. if idx = -1 or ngrid + 2, wraps it back inside [0, ngrid-1].
    return idx % ngrid #12 % 10 = 2	Index wraps around: cell 12 → cell 2; 9 % 10 = 9	still inside box
#a%b: % is the modulo operator, which gives the REMAINDER when a is divided by b.... 10%3 = 10/3=3 (3x3=9) and remainder is 1, so 10%3 = 1.

def apply_periodic_pos(pos, ngrid): #periodic boundary conditions to CONTINUOUS positions in grid units. if pos = [ngrid + 0.5], wraps to [0.5].
    return np.mod(pos, ngrid) #10.3	mod = 0.3 -> 10.3 wraps to 0.3
#np.mod: for each number in pos (i.e. 10.3), compute its remainder after dividing by ngrid (i.e.@pos 200 th-cell % 128 grid cells ).

def choose_random_direction(): 
    theta = np.arccos(np.random.uniform(-1, 1))
    phi = np.random.uniform(0, 2 * np.pi)
    return np.array([np.sin(theta) * np.cos(phi),
                     np.sin(theta) * np.sin(phi),
                     np.cos(theta)], dtype=np.float64)

#def launching_rays(ray_centre_location, random_direction_vector, ionized_mask, max_step_size, step_size=0.05, max_steps = 1000): 
def launching_rays(ray_centre_location, random_direction_vector, ionized_mask, ngrid, cell_size, max_step_size, step_size=0.05, max_steps=1000):
    pos = np.array(ray_centre_location, dtype=np.float64)
    for _ in range(max_steps):
        pos += random_direction_vector * step_size
        pos = apply_periodic_pos(pos, ngrid) #continuity position boundary condition
        idx = np.floor(pos).astype(int) #discrete index boundary condition (associated index cell for vector-position travelled)
        idx = apply_periodic_index(idx, ngrid)
        if not ionized_mask[tuple(idx)]:
            return step_size * _ * cell_size #0.05 x ith x 2cMpc/h = total distance travelled
    return max_step_size #step_size * max_steps * cell_size #if it never hits a neutral regions, it means fully ionized, then take the full box size

#loop over parameters
for i, log10Mmin in enumerate(list_log10Mmin):
    fcoll_arr = matter_fields.get_fcoll_for_Mmin(log10Mmin)
    for j, zeta in enumerate(list_zeta):
        ionization_map = script.ionization_map(matter_fields)
        qi_arr = ionization_map.get_qi(zeta * fcoll_arr)

        ionized_mask = (qi_arr > 0.5) #SET IONIZATION FRACTION 
        grid_shape = qi_arr.shape
        box_size = default_simulation_data.box
        ngrid = grid_shape[0] #128 cells in each dimension
        cell_size = box_size / ngrid #256 cMpc/h / 128 cells = 2 cMpc/h per cell = resolution

        mean_free_paths = []
        ionized_coords = np.argwhere(ionized_mask)
        num_iterations = 3000 #reduce for speed, increase for accuracy

        for _ in range(num_iterations):
            if len(ionized_coords) == 0:
                break
        
            ray_centre_idx = ionized_coords[np.random.choice(len(ionized_coords))] #randomly choose a cell in the ionized region
            direction = choose_random_direction()
            mfp = launching_rays(ray_centre_idx, direction, ionized_mask, ngrid, cell_size, box_size)
            #mfp = launching_rays(ray_centre_idx, direction, ionized_mask, box_size)
            mean_free_paths.append(mfp)

        mfp_array = np.array(mean_free_paths)
        counts, bin_edges = np.histogram(mfp_array, bins=20, density=False)
        bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])
        bin_widths = bin_edges[1:] - bin_edges[:-1]
        r_times_dN_dR = bin_centers * counts / bin_widths

        ax = axes[i, j]
        ax.plot(bin_centers, r_times_dN_dR, drawstyle='steps-mid', color='green')
        ax.set_title(rf"$\log_{{10}}(M_{{\min}})={log10Mmin},\ \zeta={zeta}$", fontsize=14, pad=20)
        ax.set_xlabel("R (cMpc/h)", fontsize=14)
        ax.set_xlim(0, default_simulation_data.box * 1.1)
        ax.set_ylabel(r"$R\,dN/dR$", fontsize=14)
        ax.tick_params(labelsize=12)

#plt.savefig('/Users/sophiatonelli/Desktop/pngs_mpia/mfp_7to11M_10to40epsilon_redshift8.png', dpi=300, bbox_inches='tight')
plt.show()
